# BasicTick: Delete Everything


In [1]:
import os
import subprocess
import boto3
import json
import datetime

from managed_kx import *
from env import *
import pykx as kx

# Cluster names and database
from basictick_setup import *

In [2]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    os.system(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


## Delete all resources

### Killing FeedHandler
You can connect to the running feedhandler and have it exit.

```
$ q
q)c:hopen ":localhost:5030"
q)c "exit 0"
```

Below we do the equivalent from Python

In [3]:
print("Stopping local FeedHandler ...")

# Stop the FeedHandler
try:
  fh=kx.QConnection(port=FH_PORT)
  fh("exit 0") 
  print("Stopped")
except:
  print("An exception occurred, already stopped")

print("Deleting Clusters...")

# Clusters
for k,v in all_clusters.items():
    print(f"\tDelete: {v}")
    try:
        client.delete_kx_cluster(environmentId=ENV_ID, clusterName=v)
        print(f"\tDeleted: {v}")
    except:
        print(f"\tAlready Deleted? {v}")
        next

print("** Deleted Clusters **")


Stopping local FeedHandler ...
An exception occurred, already stopped
Deleting Clusters...
	Delete: TP_basictickdb
	Deleted: TP_basictickdb
	Delete: RDB_basictickdb
	Deleted: RDB_basictickdb
	Delete: HDB_basictickdb
	Deleted: HDB_basictickdb
	Delete: GATEWAY_basictickdb
	Already Deleted? GATEWAY_basictickdb
	Delete: CALC_basictickdb
	Deleted: CALC_basictickdb
** Deleted Clusters **


In [4]:
# Dataview
print(f"Deleting Dataview: {DBVIEW_NAME}...")
try:
    client.delete_kx_dataview(environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME)
    wait_for_dataview_status(client=client, environmentId=ENV_ID, databaseName=DB_NAME, dataviewName=DBVIEW_NAME, status='DELETED', show_wait=True)
    print("** Deleted View **")
except:
    pass

# Volume
print(f"Deleting Volume: {VOLUME_NAME}...")
try:
    client.delete_kx_volume(environmentId=ENV_ID, volumeName=VOLUME_NAME)
    wait_for_volume_status(client=client, environmentId=ENV_ID, volumeName=VOLUME_NAME, status='DELETED', show_wait=True)
    print("** Deleted Volume **")
except:
    print(f"Already Deleted? {VOLUME_NAME}...")
    pass

# Scaling Group
print(f"Deleting Scaling Group: {SCALING_GROUP_NAME}...")
try:
    client.delete_kx_scaling_group(environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME)
    wait_for_scaling_group_status(client=client, environmentId=ENV_ID, scalingGroupName = SCALING_GROUP_NAME, status='DELETED', show_wait=True)
    print("** Deleted Scaling Group **")
except:
    print(f"Already Deleted? {SCALING_GROUP_NAME}...")
    pass

# database
print(f"Deleting Database: {DB_NAME}...")
try:
    client.delete_kx_database(environmentId=ENV_ID, databaseName=DB_NAME)    
    print("** Deleted Database **")
except:
    pass

print("\nDone Deleting")

Deleting Dataview: basictickdb_DBVIEW...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:00:00, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:00:30, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:01:00, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:01:30, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:02:00, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:02:30, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:03:00, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:03:30, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:04:00, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:04:30, waiting 30 sec ...
Dataview: basictickdb_DBVIEW status is DELETING, total wait 0:05:00, waiting 30

In [5]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2024-04-17 12:54:31.757770
